In [2]:
# tweepy is used to call the Twitter API from Python
import tweepy
import re
import credentials as cr
 
# Authenticate to Twitter API
auth = tweepy.OAuthHandler(cr.TWITTER_CONSUMER_KEY, cr.TWITTER_CONSUMER_SECRET)
auth.set_access_token(cr.TWITTER_ACCESS_TOKEN_KEY, cr.TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)